<a href="https://colab.research.google.com/github/seanvw/TR_Trade_1/blob/main/Share_dealing_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exploration of newer OO Python features

*   Need to think about Weighted Mean Share Holding Time when buying and selling shares for APR type calculations
*   Need to generate ShareReport objects for Portfolio Summary
*  



In [69]:
from functools import cached_property
import yfinance as yf
from datetime import datetime
from dataclasses import dataclass

# changed to data classes

@dataclass
class Transaction():
    quantity: float
    datetime: datetime

    def __post_init__(self):
        pass
        #print("Transactions's __post_init__")

@dataclass
class BuyTransaction(Transaction):
    buy_price: float

    def __post_init__(self):
        #print("BuyTransaction __post_init__")
        super().__post_init__()

    @property
    def invested(self):
        return self.buy_price * self.quantity

    def __str__(self) -> str:
      print(f"BuyTransaction(buy_price={self.buy_price}, \
quantity={self.quantity}, invested={self.invested}, \
buy_datetime={self.datetime}")
      return "+++"

@dataclass
class SellTransaction(Transaction):
    buy_price: float
    sell_price: float
    tax: float

    def __post_init__(self):
        #print("SellTransaction __post_init__")
        super().__post_init__()

    @cached_property
    def profit_before_tax(self) -> float:
        return (self.sell_price - self.buy_price) * self.quantity

    @cached_property
    def profit_after_tax(self) -> float:
        return self.profit_before_tax - self.tax

    @cached_property
    def tax_rate(self) -> float:
        return self.tax / self.profit_before_tax

    @cached_property
    def roi_after_tax(self) -> float:
        return self.profit_after_tax / (self.buy_price * self.quantity) * 100

    @cached_property
    def roi_before_tax(self) -> float:
        return self.profit_before_tax / (self.buy_price * self.quantity) * 100


    def __str__(self) -> str:
      print(f"SellTransaction(buy_price={self.buy_price}, sell_price={self.sell_price}, \
      quantity={self.quantity}, tax={self.tax}, sell_datetime={self.datetime}, \
      profit_before_tax={self.profit_before_tax}, profit_after_tax={self.profit_after_tax}, \
      tax_rate={self.tax_rate}, roi_after_tax={self.roi_after_tax: .2f}, roi_before_tax={self.roi_before_tax: .2f})")
      return "---"

@dataclass
class CurrentHolding():
    buy_price: float
    quantity: float
    last_transaction_datetime: datetime

    @property
    def current_invested(self):
        return self.buy_price * self.quantity

    def __str__(self) -> str:
      current_invested = self.current_invested
      print(f"CurrentHolding(buy_price={self.buy_price}, quantity={self.quantity}, \
current_invested={current_invested}, last_transaction_datetime={self.last_transaction_datetime}")
      return "==="

@dataclass
class ExchangeRate_USD_EUR():

  def __post_init__(self):
        stock = yf.Ticker("USDEUR=X")
        self.usd_eur = stock.info['regularMarketPrice']

  def get_usd_eur(self):
    return self.usd_eur

@dataclass
class Share():
    ticker: str
    usd_eur: float
    transactionsLog: list = None
    CurrentHolding: CurrentHolding = None
    current_eur_price: float = None

    def add_transaction(self, buy_price, quantity, buy_datetime):

        buy_t = BuyTransaction(buy_price=buy_price, quantity=quantity, datetime=buy_datetime)
        if self.transactionsLog is None:
          self.transactionsLog = []

        self.transactionsLog.append(buy_t)

        if self.CurrentHolding is None:
          self.CurrentHolding = CurrentHolding(buy_price=buy_t.buy_price, quantity=buy_t.quantity, last_transaction_datetime=buy_t.datetime)
        else:
          self.CurrentHolding.buy_price = (self.CurrentHolding.buy_price * self.CurrentHolding.quantity
            + buy_t.buy_price * buy_t.quantity) / (self.CurrentHolding.quantity + buy_t.quantity)
          self.CurrentHolding.quantity += buy_t.quantity
          self.CurrentHolding.last_transaction_datetime = buy_t.datetime

    def sell_transaction(self, sell_price, quantity, tax, sell_datetime):

        sell_t = SellTransaction(buy_price=self.CurrentHolding.buy_price,
                                 sell_price=sell_price, quantity=quantity,
                                 tax=tax, datetime=sell_datetime)
        # update the current holding
        self.CurrentHolding.quantity -= sell_t.quantity
        self.CurrentHolding.last_transaction_datetime = sell_t.datetime
        self.transactionsLog.append(sell_t)

    @cached_property
    def get_current_price(self) -> float:
      stock = yf.Ticker(self.ticker)
      if self.current_eur_price is None:
        price_usd = stock.info['regularMarketPrice']  # Get the current price
        self.current_eur_price = price_usd * self.usd_eur
      return self.current_eur_price

    def get_current_value(self) -> float:
      return self.get_current_price * self.CurrentHolding.quantity

    def get_current_roi(self) -> float:
      cp = self.get_current_price
      return (cp - self.CurrentHolding.buy_price) / self.CurrentHolding.buy_price * 100


    # this should be changed to new method
    # the new function to return an object with reporting information needed by share totaller

    def __str__(self) -> str:
        #print(f"Share ticker {self.ticker}")
        for transaction in self.transactionsLog:
          print(transaction)
        print(self.CurrentHolding)
        q = self.CurrentHolding.quantity
        print(f"Current Exchange Rate(USD->EUR): {self.usd_eur: .6f}")
        print(f"Current Share Price: {self.get_current_price: .2f} EUR")
        print(f"Current Quantity Held: {q: .2f}")
        print(f"Current Holding Value: {self.get_current_value(): .2f} EUR")
        print(f"Current Holding ROI: {self.get_current_roi(): .2f} %")
        return ""

        print(self.CurrentHolding)


@dataclass
class Portfolio():
    owner: str
    platform: str

    def __post_init__(self):
        self.shares = {}

    def __str__(self) -> str:
      print(f"------------------ Portfolio report begin ---------------\n")
      print(f"Portfolio owner: {self.owner}")
      print(f"Portfolio platform: {self.platform}\n")
      print(f"Portfolio shares:")
      for k, v in self.shares.items():
        print(f"Share: {k}")
        print(v)
      return "------------------- Portfolio report done ---------------\n"

    def add_share(self, ticker, buy_price, quantity, buy_datetime) -> Share:
        if ticker in self.shares:
          #print(f"Ticker {ticker} found in portfolio.")
          share = self.shares[ticker]
        else:
          #print(f"Ticker {ticker} not found in portfolio. Creating new share.")
          exchange_rate = ExchangeRate_USD_EUR()
          usd_eur = exchange_rate.get_usd_eur()
          share = Share(ticker,usd_eur)
          self.shares[ticker] = share

        share.add_transaction(buy_price, quantity, buy_datetime)
        #print(share)
        return share


    def sell_share(self, ticker, sell_price, quantity, tax, sell_datetime) -> Share:
        if ticker in self.shares:
          share = self.shares[ticker]
          #print(f"sell_share, Ticker {ticker} found in portfolio.")
          share.sell_transaction(sell_price, quantity, tax, sell_datetime)
          return(share)
        else:
          raise ValueError(f"Ticker {ticker} not found in portfolio.")


    def generate_report(self):
      pass



In [70]:
portfolio = Portfolio(owner='SW', platform='Trade Republic')

portfolio.add_share(ticker='AAPL', buy_price=160.0, quantity=1.5, buy_datetime=datetime(2023, 10, 1, 9, 30))
print(portfolio)

portfolio.add_share(ticker='AAPL', buy_price=200.0, quantity=1.5, buy_datetime=datetime(2023, 10, 2, 10, 30))
print(portfolio)

portfolio.sell_share(ticker='AAPL', sell_price=240, quantity=1, tax=15, sell_datetime=datetime(2023, 11, 1, 14, 30))
print(portfolio)

portfolio.add_share(ticker='GOOG', buy_price=200.0, quantity=10, buy_datetime=datetime(2023, 12, 2, 10, 30))
print(portfolio)


------------------ Portfolio report begin ---------------

Portfolio owner: SW
Portfolio platform: Trade Republic

Portfolio shares:
Share: AAPL
BuyTransaction(buy_price=160.0, quantity=1.5, invested=240.0, buy_datetime=2023-10-01 09:30:00
+++
CurrentHolding(buy_price=160.0, quantity=1.5, current_invested=240.0, last_transaction_datetime=2023-10-01 09:30:00
===
Current Exchange Rate(USD->EUR):  0.916400
Current Share Price:  199.74 EUR
Current Quantity Held:  1.50
Current Holding Value:  299.61 EUR
Current Holding ROI:  24.84 %

------------------- Portfolio report done ---------------

------------------ Portfolio report begin ---------------

Portfolio owner: SW
Portfolio platform: Trade Republic

Portfolio shares:
Share: AAPL
BuyTransaction(buy_price=160.0, quantity=1.5, invested=240.0, buy_datetime=2023-10-01 09:30:00
+++
BuyTransaction(buy_price=200.0, quantity=1.5, invested=300.0, buy_datetime=2023-10-02 10:30:00
+++
CurrentHolding(buy_price=180.0, quantity=3.0, current_invested=

**----------------------------------------- legacy ---------------------------**

In [ ]:


######################### older style  before refactor #########################
class Portfolio_OLD():
    def __init__(self,owner,platform):
        self.shares = {}
        self.owner = owner
        self.platform = platform

    def __str__(self) -> str:
      print(f"------------------ Portfolio report begin ---------------\n")
      print(f"Portfolio owner: {self.owner}")
      print(f"Portfolio platform: {self.platform}\n")
      print(f"Portfolio shares:")
      for k, v in self.shares.items():
        print(f"Share: {k}")
        print(v)
      return "------------------- Portfolio report done ---------------\n"

    def add_share(self, ticker, buy_price, quantity, buy_datetime) -> Share:
        if ticker in self.shares:
          #print(f"Ticker {ticker} found in portfolio.")
          share = self.shares[ticker]
        else:
          #print(f"Ticker {ticker} not found in portfolio. Creating new share.")
          exchange_rate = ExchangeRate_USD_EUR()
          usd_eur = exchange_rate.get_usd_eur()
          print(f"USD to EUR rate: {usd_eur}")
          share = Share(ticker,usd_eur)
          self.shares[ticker] = share

        share.add_transaction(buy_price, quantity, buy_datetime)
        #print(share)
        return share


    def sell_share(self, ticker, sell_price, quantity, tax, sell_datetime) -> Share:
        if ticker in self.shares:
          share = self.shares[ticker]
          #print(f"sell_share, Ticker {ticker} found in portfolio.")
          share.sell_transaction(sell_price, quantity, tax, sell_datetime)
          return(share)
        else:
          raise ValueError(f"Ticker {ticker} not found in portfolio.")


    def generate_report(self):
      pass



class Transaction_OLD():
    def __init__(self, quantity, datetime):
        self.quantity = quantity
        self.datetime = datetime

# old style call
# positional args
# buy_t = BuyTransaction(buy_price, quantity, buy_datetime)

class BuyTransaction_OLD(Transaction):
    def __init__(self, buy_price, quantity, buy_datetime):
        super().__init__(quantity, buy_datetime)
        self.buy_price = buy_price
        self.invested = buy_price * quantity

    def __str__(self) -> str:
      print(f"BuyTransaction(buy_price={self.buy_price}, \
quantity={self.quantity}, invested={self.invested}, \
buy_datetime={self.datetime}")
      return "+++"

# old style call
#sell_t = SellTransaction(self.CurrentHolding.buy_price, sell_price, quantity, tax, sell_datetime)

class SellTransaction_OLD(Transaction):
    def __init__(self, buy_price, sell_price, quantity, tax, sell_datetime):
        super().__init__(quantity, sell_datetime)
        self.buy_price = buy_price
        self.sell_price = sell_price
        self.tax = tax
        self.profit_before_tax = (sell_price - buy_price) * quantity
        self.profit_after_tax = self.profit_before_tax - tax
        self.tax_rate = tax / self.profit_before_tax
        self.roi_after_tax = self.profit_after_tax / (buy_price * quantity) * 100
        self.roi_before_tax = self.profit_before_tax / (buy_price * quantity) * 100

    def __str__(self) -> str:
      print(f"SellTransaction(buy_price={self.buy_price}, sell_price={self.sell_price}, \
quantity={self.quantity}, tax={self.tax}, sell_datetime={self.datetime}, \
profit_before_tax={self.profit_before_tax}, profit_after_tax={self.profit_after_tax}, \
tax_rate={self.tax_rate}, roi_after_tax={self.roi_after_tax: .2f}, roi_before_tax={self.roi_before_tax: .2f})")
      return "---"

# old style call
# self.CurrentHolding = CurrentHolding(buy_t.buy_price, buy_t.quantity, buy_t.datetime)

class CurrentHolding_OLD():
    def __init__(self, buy_price, quantity, last_transaction_datetime):
        self.buy_price = buy_price
        self.quantity = quantity
        self.last_transaction_datetime = last_transaction_datetime

    def get_current_invested(self):
      return self.buy_price * self.quantity

    def __str__(self) -> str:
      current_invested = self.get_current_invested()
      print(f"CurrentHolding(buy_price={self.buy_price}, quantity={self.quantity}, \
current_invested={current_invested}, last_transaction_datetime={self.last_transaction_datetime}")
      return "==="



class Share_OLD():
    def __init__(self, ticker, usd_eur):
        self.ticker: str = ticker
        self.transactionsLog: list = []
        self.CurrentHolding: CurrentHolding = None
        self.usd_eur: float = usd_eur
        self.current_eur_price: float = None

    def add_transaction(self, buy_price, quantity, buy_datetime):

        buy_t = BuyTransaction(buy_price=buy_price, quantity=quantity, datetime=buy_datetime)
        self.transactionsLog.append(buy_t)
        if self.CurrentHolding is None:
          self.CurrentHolding = CurrentHolding(buy_price=buy_t.buy_price, quantity=buy_t.quantity, last_transaction_datetime=buy_t.datetime)
        else:
          self.CurrentHolding.buy_price = (self.CurrentHolding.buy_price * self.CurrentHolding.quantity
            + buy_t.buy_price * buy_t.quantity) / (self.CurrentHolding.quantity + buy_t.quantity)
          self.CurrentHolding.quantity += buy_t.quantity
          self.CurrentHolding.last_transaction_datetime = buy_t.datetime

    def sell_transaction(self, sell_price, quantity, tax, sell_datetime):

        sell_t = SellTransaction(buy_price=self.CurrentHolding.buy_price, sell_price=sell_price, quantity=quantity, tax=tax, datetime=sell_datetime)
        # update the current holding
        self.CurrentHolding.quantity -= sell_t.quantity
        self.CurrentHolding.last_transaction_datetime = sell_t.datetime
        self.transactionsLog.append(sell_t)

    def get_current_price(self) -> float:
      stock = yf.Ticker(self.ticker)
      if self.current_eur_price is None:
        price_usd = stock.info['regularMarketPrice']  # Get the current price
        self.current_eur_price = price_usd * self.usd_eur
      return self.current_eur_price

    def get_current_value(self) -> float:
      return self.get_current_price() * self.CurrentHolding.quantity

    def get_current_roi(self) -> float:
      cp = self.get_current_price()
      return (cp - self.CurrentHolding.buy_price) / self.CurrentHolding.buy_price * 100


    # this should be changed to new method
    # the new function to return an object with reporting information needed by share totaller

    def __str__(self) -> str:
        #print(f"Share ticker {self.ticker}")
        for transaction in self.transactionsLog:
          print(transaction)
        print(self.CurrentHolding)
        q = self.CurrentHolding.quantity
        print(f"Current Exchange Rate(USD->EUR): {self.usd_eur: .6f}")
        print(f"Current Share Price: {self.get_current_price(): .2f} EUR")
        print(f"Current Quantity Held: {q: .2f}")
        print(f"Current Holding Value: {self.get_current_value(): .2f} EUR")
        print(f"Current Holding ROI: {self.get_current_roi(): .2f} %")
        return ""

        print(self.CurrentHolding)


######################### old style END ######################
